In [0]:
import pandas as pd

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
path = 'https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/queens.csv'
df = pd.read_csv(path)
y = df.rent
X = df.drop('rent', axis=1).select_dtypes('number')

In [0]:
def test_pred(y_pred, y_test):
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    for metric in  mean_absolute_error, mean_squared_error, r2_score:
         print(f'{metric.__name__}: {metric(y_pred, y_test)}')

In [0]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import r2_score
import numpy as np

def _closest_k(X1, X2, k=5):
    matrix = euclidean_distances(X1, X2)
    df = pd.DataFrame(matrix,
                 index=X1.index, 
                 columns=X2.index)
    
    order = np.argsort(matrix, axis=1)[:, :k]
    result = pd.DataFrame(X2.index[order], 
                      columns=np.arange(k),
                      index=X1.index)
    
    return result, df

In [0]:
class KNN:
    X = None
    y = None
    
    def __init__(self, k=5):
        self.k = k
    
    def fit(self, X, y):
        self.X = X
        self.y = y
    

    def predict(self, X):
        closest, matrix = _closest_k(X, self.X, self.k)
        
        s1 = self.y[closest.iloc[:, 0]]
        s2 = []
        for e in matrix.index:
            s2.append(matrix.loc[e, closest.loc[e, 0]] ** 2)
        s2 = np.array(s2)
        
        s1 *= s2
        
        for i in range(1, self.k):
            s1_ = self.y[closest.iloc[:, i]].values.astype('float64')
            
            s2_ = []
            for e in matrix.index:
                s2_.append(matrix.loc[e, closest.loc[e, i]] ** 2)
            s2 += np.array(s2_)
            
            s1_ *= s2_
            s1 += s1_
        
        s1.index = X.index
        return s1 / s2
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y_pred, y)

# Fit Model

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=1324)

In [0]:
scaler = MinMaxScaler()
# scaler.fit(X_train)

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns, index=X_test.index)

In [0]:
model = KNN(k=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [0]:
test_pred(y_pred, y_test)

mean_absolute_error: 375.0551472722361
mean_squared_error: 262747.1031584342
r2_score: 0.4139216831786259


In [0]:
"""
Лучший результат получается при 4-5 K


Самый простой способ - нормализовать данные с помощью MinMaxScaler
Как уже говорилось, выделять значимые признаки
Кодировать категориальные признаки


Долго считает евклидово расстояние между всеми признаками


Плюсы:
Простая реализация
Устойчив к выбросам, т.к. шанс попадания выброса в k ближайших соседей очень мал.
Минусы:
Хранит в себе весь тестовый набор, а значит требует много памяти


"""